# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [13]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [14]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("../data/books_infos.csv", index_col=0)

# Afficher les informations sur le DataFrame
print(df_books.info())

# Vérifier le type de df_books
print(df_books.describe())



<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1000 non-null   object
 1   price         1000 non-null   object
 2   rating        1000 non-null   object
 3   availability  1000 non-null   object
dtypes: object(4)
memory usage: 39.1+ KB
None
                         title   price rating availability
count                     1000    1000   1000         1000
unique                     999     903      5            1
top     The Star-Touched Queen  £27.88    One     In stock
freq                         2       3    226         1000


## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [3]:
# Vérifier le type des colonnes
print(df_books.dtypes)

title           object
price           object
rating          object
availability    object
dtype: object


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [4]:
# # Conversion de title en chaîne de caractères
# df_books["title"] = df_books["title"].astype(str)

# # Vérification du type de la colonne title
# print(df_books["title"].dtype)

Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [5]:
# # Convertir la colonne price en type décimal avec étape intermédiaire pour retirer le caractère "£"
# df_books["price"] = df_books["price"].str.replace("£", "", n=-1).astype(float)

# # Vérification du type de la colonne price
# print(df_books["price"].dtype)

Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

On a une valeur 'In stock' et même si on a aucune autre valeur, on peut supposer une valeur 'Out of stock'.
On peut donc convertir ça en booléan avec 'In stock' = True et le reste = False.

In [6]:
# # Valeurs possibles de la colonne availability
# df_books["availability"].unique()
# df_books["availability"].value_counts()


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [7]:
# # Fonction pour convertir la valeur de availability en booléen
# def convert_availability(value : str) -> bool:
#     """Convert the availability value to a boolean.

#     Args:
#         value (str): The availability status of the book.

#     Returns:
#         bool: True if the book is available, False otherwise.
#     """
#     if value == 'In stock':
#         return True
#     return False
     

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [8]:
# # Convertir la colonne availability en booléen (True/False)
# df_books["availability"] = df_books["availability"].apply(convert_availability)

# # Vérification du type de la colonne availability
# print(df_books["availability"].dtype)

Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [9]:
# # Nettoyage de 'rating' avant mapping
# df_books["rating"] = df_books["rating"].astype(str).str.strip().str.title()

# # Dictionnaire associant les notes au format initial et les valeurs numérique
# ratings_map = {
#     'One': 1,
#     'Two': 2,
#     'Three': 3,
#     'Four': 4,
#     'Five': 5
# }

# df_books["rating"] = df_books["rating"].map(ratings_map)

# # Vérification du type de la colonne rating
# print(df_books["rating"].dtype)

In [10]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """

    # Convertir la colonne title en chaîne
    df_books["title"] = df_books["title"].astype(str)

    # Nettoyer le prix (retirer £ et convertir en float)
    df_books["price"] = df_books["price"].astype(str).str.replace("£", "", regex=False)
    df_books["price"] = df_books["price"].astype(float)

    # Convertir la disponibilité en booléen
    def convert_availability(value):
        if value == "In stock":
            return True
        return False
    
    df_books["availability"] = df_books["availability"].apply(convert_availability)

    # Nettoyage de 'rating' avant mapping
    df_books["rating"] = df_books["rating"].astype(str).str.strip().str.title()

    # Convertir les notes textuelles en notes numériques
    ratings_map = {
        "One": 1,
        "Two": 2,
        "Three": 3,
        "Four": 4,
        "Five": 5,
    }
    df_books["rating"] = df_books["rating"].map(ratings_map)

    return df_books

print(convert_types(df_books).dtypes) 

print(df_books.head())


title            object
price           float64
rating            int64
availability       bool
dtype: object
                                   title  price  rating  availability
0                   A Light in the Attic  51.77       3          True
1                     Tipping the Velvet  53.74       1          True
2                             Soumission  50.10       1          True
3                          Sharp Objects  47.82       4          True
4  Sapiens: A Brief History of Humankind  54.23       5          True


---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [11]:
# Création de la BDD et insertion des données

# Création de la BDD books_infos.db dans le dossier database
connection = sqlite3.connect("../database/book_store.db")

# On vérifie que la BDD est créée
print(connection.total_changes)

# On crée une table dans la BDD avec le df
df_books.to_sql('book_store', connection, if_exists='replace')

0


1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [12]:
# Compter le nombre de livre dans la BDD
# Création d'un curseur pour interagir avec la DB
cursor = connection.cursor()

# Exécuter la requête pour compter le nombre de livre dans la DB
cursor.execute("SELECT COUNT(*) FROM book_store")

# Affichage du résultat
print(cursor.fetchone()[0])

# # On déconnecte la DB
# connection.close()

1000
